In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [29]:
 #Fungsi untuk menghitung Entropy
def entropy(y):
    #Mehitung jumlah kemunculan kelas, hasil berupa array [a, b]
    kemunculan_kelas = np.bincount(y)
    #Menghitung probabilitas kemunculan kelas, hasil berupa array [a, b]
    prob_kemunculan = kemunculan_kelas/len(y)
    #Return rumus entropy, dengan kondisi px tidak kurang dari 0
    return -np.sum([px * np.log2(px) for px in prob_kemunculan if px > 0])

#Kelas pembantu untuk menyimpan informasi node
class Node:
    #Parameter dari Node yang ada pada sebuah Decision Tree yang nilainya perlu disimpan
    #* digunakan untuk keyword only parameter
    def __init__(self, fitur=None, threshold=None, node_kiri=None, node_kanan=None, *, nilai_leaf=None):
        self.fitur = fitur
        self.threshold = threshold
        self.node_kiri = node_kiri
        self.node_kanan = node_kanan
        self.nilai_leaf = nilai_leaf
    
    #Fungsi pembantu untuk mengindikasikan bawah proses spilitting sudah sampai di node leaf
    def leaf_node(self):
        return self.nilai_leaf is not None
    
#Kelas Decision Tree
class DecisionTree:
    #Constructor dengan parameter untuk proses splitting
    def __init__(self, min_samples_split=2, max_depth=100, jumlah_fitur=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.jumlah_fitur = jumlah_fitur
        self.root = None
    
    #Fungsi untuk melakukan training 
    def fit(self, X, y):
        #Jumlah fitur = jumlah kolom dari x yang dipilih
        #Dengan kondisi jika nanti jumlah_fitur tidak didefinisikan maka jumlah fitur yang dipilih adalah jumlah maksimal dari fitur yang dipilih
        #Dan jumlah fitur yang dipilih tidak akan lebih besar dari jumlah_fitur yang ada
        self.jumlah_fitur = X.shape[1] if not self.jumlah_fitur else min(self.jumlah_fitur, X.shape[1])
        #Memulai pembentukan decision tree dari root
        self.root = self.tree(X, y)
    
    #Fungsi untuk membentuk decision tree
    def tree(self, X, y, depth=0):
        #Mendapatkan nilai dari jumlah sample/baris dan nilai dari jumlah fitur/kolom yang ada pada data hasil pre-processing
        n_samples, n_fitur = X.shape
        #Mendapatkan nilai dari total kelas yang ada pada data hasil pre-processing
        n_kelas = len(np.unique(y))
        
        #Stopping criteria untuk proses splitting
        #Jika kondisi terpenuhi
        if (depth >= self.max_depth
                or n_kelas == 1
                or n_samples < self.min_samples_split):
            #Maka proses splitting sudah sampai di leaf , dan nilai pada leaf bisa didapatkan menggunakan fungsi nilai_kelas_terbanyak
            nilai_leaf_node = self.nilai_kelas_terbanyak(y)
            #Menyimpan nilai leaf pada leaf_node
            return Node(nilai_leaf=nilai_leaf_node)
        
       
        #Memilih index secara random pada array n_fitur sejumlah jumlah_fitur tanpa pengembalian
        fitur_dipilih = np.random.choice(n_fitur, self.jumlah_fitur, replace=False)
        
        #Mencari fitur terbaik dan threshold terbaik dari fitur yang telah dipilih secara random
        fitur_terbaik, threshold_terbaik = self.feature_extraction(X, y, fitur_dipilih)
        
        #Membuat decision node berdasarkan rule (attribut dan threshold terbaik) yang telah dihitung melalui beberapa proses
        #Memilih semua sample data pada fitur_terbaik
        split_kiri, split_kanan = self.split(X[:, fitur_terbaik], threshold_terbaik)
        #Memilih x dan y yang ada pada sample kiri untuk dijadikan sebagai sub decision node ( sudah dengan fitur dan threshold terbaik)
        node_kiri = self.tree(X[split_kiri, :], y[split_kiri], depth+1)
        #Memilih x dan y yang ada pada sample kanan  untuk dijadikan sebagai sub decision node ( sudah dengan fitur dan threshold terbaik)
        node_kanan = self.tree(X[split_kanan, :], y[split_kanan], depth+1)
        #Menyimpan nilai node
        return Node(fitur_terbaik, threshold_terbaik, node_kiri, node_kanan)
        
    #Fungsi untuk mendapatkan fitur dan threshold terbaik
    def feature_extraction(self, X, y, fitur_dipilih):
        best_gain = -1
        split_fitur, split_threshold = None, None
        
        for fitur_index in fitur_dipilih:
            #Memilih kolom dari fitur yang dipilih
            kolom_X = X[:, fitur_index]
            #Mendapatkan nilai fitur yang dipilih/threshold secara unik
            thresholds = np.unique(kolom_X)
            for threshold in thresholds:
                #Melakukan perhitungan entropy dan information gain untuk mendapatkan best threshold pada fungsi information_gain
                gain = self.information_gain(y, kolom_X, threshold)
                
                #Kondisi untuk mendapatkan fitur dan threshold terbaik untuk dijadikan sebagai rule pada decision node
                if gain > best_gain:
                    best_gain = gain
                    split_fitur = fitur_index
                    split_threshold = threshold
                    
        return split_fitur, split_threshold
        
    #Fungsi untuk menghitung information gain
    def information_gain(self, y, kolom_X, split_threshold):
        #Menghitung parent entropy
        parent_entropy = entropy(y)
        
        #Melakukan splitting dengan fungsi split
        split_kiri, split_kanan = self.split(kolom_X, split_threshold)
        
        if len(split_kiri) == 0 or len(split_kanan) == 0:
            return 0
        
        #Menghitung child entropy
        total_kelas = len(y)
        total_kelas_kiri = len(split_kiri)
        total_kelas_kanan = len(split_kanan)
        entropy_split_kiri = entropy(y[split_kiri])
        entropy_split_kanan = entropy(y[split_kanan])
        child_entropy = (total_kelas_kiri / total_kelas) * entropy_split_kiri + (total_kelas_kanan / total_kelas) * entropy_split_kanan
        
        #Information Gain
        ig = parent_entropy - child_entropy
        return ig
        
    #Fungsi untuk splitting    
    def split(self, kolom_X, split_threshold):
        split_kiri = np.argwhere(kolom_X <= split_threshold).flatten()
        split_kanan = np.argwhere(kolom_X > split_threshold).flatten()
        return split_kiri, split_kanan
        
        
    #Fungsi untuk mendapatkan nilai kelas terbanyak pada leaf node
    def nilai_kelas_terbanyak(self, y):
        #Menghitung jumlah kemunculan kelas pada leaf node
        counter = Counter(y)
        #Mendapatkan nilai kemunculan kelas terbanyak
        kemunculan_kelas_terbanyak = counter.most_common(1)[0][0]
        #Menyimpan nilai tersebut untuk di pass
        return kemunculan_kelas_terbanyak
    
    #Fungsi untuk melakukan prediksi
    def predict(self, X):
        #Memasukan satu per satu data testing yang ada pada input data ke dalam decision tree dimulai dari root node
        return np.array([self.pergerakan_tree(x, self.root) for x in X])
    
    def pergerakan_tree(self, x, node):
        #Jika sampai di leaf node nilai_leaf didapatkan
        if node.leaf_node():
            return node.nilai_leaf
        #Jika belum akan ke kiri jika threshold data testing <= threshold rule
        if x[node.fitur] <= node.threshold:
            return self.pergerakan_tree(x, node.node_kiri)
        #Ke kanan jika threshold data testing > threshold rule
        return self.pergerakan_tree(x, node.node_kanan)

In [30]:
def bootstrapped_dataset(X, y):
        #Mendapatkan baris data X
        sampel_data = X.shape[0]
        #Memilih Index secara random dari array sampel data sejumlah sampel_data dengan pengembalian
        bootstrap_data = np.random.choice(sampel_data, size=sampel_data, replace=True)
        #Menyimpan nilai yang dipilih
        return X[bootstrap_data], y[bootstrap_data]
    
def voting(y):
        #Menghitung kemunculan kelas
        counter = Counter(y)
        #Mendapatkan nilai kemunculan kelas terbanyak
        kemunculan_kelas_terbanyak = counter.most_common(1)[0][0]
        #Menyimpan nilai tersebut untuk di pass
        return kemunculan_kelas_terbanyak
    
class RandomForest:
    #self untuk mengakses attribut dan fungsi yang ada di dalam kelas
    def __init__(self, jumlah_tree, min_samples_split=2, max_depth=100, jumlah_fitur=None):
        self.jumlah_tree = jumlah_tree
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.jumlah_fitur = jumlah_fitur
        self.array_tree = []
        
    def fit(self, X, y):
        #Melakukan training terhadap setiap decision dari sejumlah n decision tree berdasarkan bootstrapped dataset
        for single_tree in range(self.jumlah_tree):
            # Memanggil kelas Decision tree
            tree = DecisionTree(min_samples_split=self.min_samples_split, max_depth=self.max_depth, jumlah_fitur=self.jumlah_fitur)
            # memasukan nilai x dan y bootsrap data ke dalam variabel
            X_subset, y_subset = bootstrapped_dataset(X, y)
            # melatih tree berdasarkan x dan y bootstrapped dataset
            tree.fit(X_subset, y_subset)
            #Menambahkan decision tree hasil training ke dalam array tree
            self.array_tree.append(tree)
            
    def predict(self, X):
        #proses prediksi berulang
        tree_prediksi = np.array([tree.predict(X) for tree in self.array_tree])
        #misal hasil prediksi ([[1, 0, 1, 0],[1, 1, 1, 1], [1, 0, 1, 0]])
        #diubah menjadi
        #([[1, 1, 1]
        #   0, 1, 0
        #   1, 1, 1
        #   0, 1, 0])
        tree_prediksi = np.swapaxes(tree_prediksi, 0, 1)
        #Melakukan voting dari n decision tree yang telah dibangun
        prediksi_y = [voting(sekumpulan_tree) for sekumpulan_tree in tree_prediksi]
        #Menyimpan nilai
        return np.array(prediksi_y)

In [31]:
data1 = pd.read_excel('DataSaham.xlsx')
data = data1.dropna()
data.head()
print(data)
print(len(data))

              Date  Open  High   Low  Close    Upper Band    Lower Band  \
19    Apr 24, 2012  3925  3925  3875   3875  4.091305e+09  3.818695e+09   
20    Apr 25, 2012  3875  3925  3875   3900  4.087802e+09  3.809698e+09   
21    Apr 26, 2012  3900  4075  3900   4050  4.085113e+09  3.804887e+09   
22    Apr 27, 2012  4050  4075  3975   4000  4.098573e+09  3.803927e+09   
23    Apr 30, 2012  4000  4025  3975   4025  4.101090e+09  3.803910e+09   
...            ...   ...   ...   ...    ...           ...           ...   
2208  Apr 27, 2021  5800  5900  5750   5900  6.042998e+09  5.662002e+09   
2209  Apr 28, 2021  5900  5900  5800   5825  6.026606e+09  5.668394e+09   
2210  Apr 29, 2021  5875  5925  5825   5875  6.029306e+09  5.673194e+09   
2211  Apr 30, 2021  5875  5875  5650   5700  6.026769e+09  5.690731e+09   
2212  May 03, 2021  5700  5725  5550   5575  6.033885e+09  5.676115e+09   

                %K            %D  Relative Strength Index    SMA5   SMA 20  \
19    3.076923e+09  2

In [32]:
ArrayData = np.array(data)
x = ArrayData[:500,5:15]
X = x.astype(float)
Y_1D = ArrayData[:500,15:16]
y_new = Y_1D.ravel()
y = y_new.astype(int)

print(X)
print(y)

[[4.09130461e+09 3.81869539e+09 3.07692308e+09 ... 1.84507207e+09
  7.35600000e+07 6.92307692e+09]
 [4.08780224e+09 3.80969776e+09 2.22222222e+09 ... 1.64469584e+09
  6.71800000e+07 7.77777778e+09]
 [4.08511274e+09 3.80488726e+09 3.00000000e+01 ... 1.39973094e+09
  1.09530000e+08 7.00000000e+01]
 ...
 [5.20718024e+09 4.71781976e+09 2.23684210e+09 ... 3.32770577e+09
  2.65910000e+08 7.76315790e+09]
 [5.16656387e+09 4.72543613e+09 3.28947368e+09 ... 3.00345086e+09
  2.78380000e+08 6.71052632e+09]
 [5.15429899e+09 4.72020101e+09 3.42105263e+09 ... 2.74377514e+09
  2.93750000e+08 6.57894737e+09]]
[0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0 1 1 1 1 1 0 1 0
 1 1 1 1 0 1 0 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 0 0 1 1 1 1 0 1 0 1
 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0
 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1 1 1 0 0 0 1 0 1
 1 0 1 0 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 1 0 1
 1 1 1 1 1 1 0 0 1 1 0 1 

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [34]:
clf = RandomForest(jumlah_tree=100, max_depth=10)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print ("Accuracy:", acc)

Accuracy: 0.64


In [35]:
rf_matrix = confusion_matrix(y_test, y_pred)
#display(rf_matrix)

true_negative = rf_matrix[0][0]
false_negative = rf_matrix[1][0]
true_positive = rf_matrix[1][1]
false_positive = rf_matrix[0][1]

accuracy = (true_negative + true_positive) / (true_negative + false_negative + true_positive + false_positive)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
#specifity = true_negative / (true_negative + false_positive)
F1_Score = 2 * (recall * precision) / (recall + precision)

print('Accuracy :',format(float(accuracy)))
print('Precision :',format(float(precision)))
print('Recall :',format(float(recall)))
#print('Specifity : ',format(float(specifity)))
print('F1 Score :',format(float(F1_Score)))

Accuracy : 0.64
Precision : 0.7021276595744681
Recall : 0.717391304347826
F1 Score : 0.7096774193548387


In [36]:
#Mengembalikan copy an array dari data y_test dan y_predict ke dalam bentuk 1 dimensi
a = y_test.flatten()
b = y_pred.flatten()
c = np.where(a==b,"TRUE","FALSE")
index_1 = X_test[:,:1]
index_1_flat = index_1.flatten()
index_2 = X_test[:,1:2]
index_2_flat = index_2.flatten()
index_3 = X_test[:,2:3]
index_3_flat = index_3.flatten()
index_4 = X_test[:,3:4]
index_4_flat = index_4.flatten()
index_5 = X_test[:,4:5]
index_5_flat = index_5.flatten()
index_6 = X_test[:,5:6]
index_6_flat = index_6.flatten()
index_7 = X_test[:,6:7]
index_7_flat = index_7.flatten()
index_8 = X_test[:,7:8]
index_8_flat = index_8.flatten()
index_9 = X_test[:,8:9]
index_9_flat = index_9.flatten()
index_10 = X_test[:,9:10]
index_10_flat = index_10.flatten()
df = pd.DataFrame({'Upper Band': index_1_flat, 'Lower Band': index_2_flat,
                   'Attribut 3': index_3_flat, 'Attribut 4': index_4_flat,
                   'Attribut 5': index_5_flat, 'Attribut 6': index_6_flat,
                   'Attribut 7': index_7_flat, 'Attribut 8': index_8_flat,
                   'Attribut 9': index_9_flat, 'Attribut 10': index_10_flat,
                   'Kelas Sebenarnya': a, 'Kelas Prediksi': b, 'Kesimpulan': c.flatten()})

print(df)

       Upper Band    Lower Band    Attribut 3    Attribut 4    Attribut 5  \
0    4.814278e+08  3.318222e+08  4.615385e+09  4.715667e+09  5.423329e+09   
1    4.820224e+09  3.344776e+09  1.025641e+09  3.628976e+08  4.811226e+09   
2    4.800734e+09  3.421766e+09  2.702703e+09  1.970432e+08  5.310021e+09   
3    4.802049e+09  3.490451e+09  2.702703e+09  1.332871e+09  5.378371e+09   
4    4.778967e+09  3.588533e+09  2.432432e+09  1.801802e+09  5.449785e+08   
..            ...           ...           ...           ...           ...   
145  5.253345e+09  4.746155e+09  2.105263e+09  1.659919e+09  3.984196e+09   
146  5.219691e+09  4.732309e+09  7.894737e+09  1.666667e+09  4.317015e+09   
147  5.207180e+09  4.717820e+09  2.236842e+09  1.710526e+09  4.636568e+08   
148  5.166564e+09  4.725436e+09  3.289474e+09  2.105263e+09  4.676861e+09   
149  5.154299e+09  4.720201e+09  3.421053e+09  2.982456e+08  4.602390e+09   

     Attribut 6  Attribut 7    Attribut 8   Attribut 9   Attribut 10  \
0  